# Data Preparation

In [10]:
from google.cloud import storage
import os
import zipfile

In [ ]:
if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists(os.path.join('data', 'bismillah')):
    os.mkdir(os.path.join('data', 'bismillah'))
if not os.path.exists('zip_files'):
    os.mkdir('zip_files')

In [ ]:
storage_client = storage.Client("[whisper-arabic-transcription]")
bucket = storage_client.get_bucket('recitation_dataset')
blobs = list(storage_client.list_blobs('recitation_dataset'))

In [14]:
output_path = os.path.join('data', 'bismillah')

for blob in blobs:
    zip_name = os.path.join('zip_files', blob.name)
    blob.download_to_filename(zip_name)
    with zipfile.ZipFile(zip_name,"r") as zip_ref:
        zip_ref.extractall(os.path.join('data', 'bismillah'))

# Training

In [ ]:
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install transformers datasets huggingsound

In [35]:
from huggingsound import TrainingArguments, ModelArguments, SpeechRecognitionModel, TokenSet
import pandas as pd
import json

In [37]:
with open("vocab.json", "r", encoding="utf8") as f:
    results = json.load(f)
    vocab = [x for x,y in results.items()]
token_set = TokenSet(vocab)

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")
output_dir = "saved_models"

In [38]:
df = pd.read_csv("final_dataset.csv", encoding="utf8")
df.rename(columns={"aayah": "transcription", "audio_paths": "path"}, inplace=True)
train_data = [df.iloc[i].to_dict() for i in range(len(df))]

In [39]:
len(train_data)

150315

In [ ]:
# and finally, fine-tune your model
model.finetune(
    output_dir, 
    train_data=train_data, 
    token_set=token_set,
)